### Huckle World
Game Setup
* 4x4 grid
* Hurkle occupies one coordinate
* User has a fixed number of guesses
* Initial guess is random


![Huckle](images/hucklew.png)




Gameplay
* If initial guess is correct:
    * Game ends immediately
    * Hurkle is "caught"
* If guess is incorrect:
    * Environment provides a directional clue
    * Clue indicates one of eight directions: N, E, W, S, NE, SE, SW, NW
    * User makes another guess based on the clue

Game Progression
* Continues until user guesses Hurkle's location correctly
* Score = number of guesses used (lower is better)
Key Points
* Hurkle knows its own location
* Only one Hurkle per game
* Clues guide the user towards Hurkle's location
* Game combines luck (initial guess) and strategy (using clues)


In our case, we are going to focus on applying  Probabilistic Logic and do three tasks, as we are not developing the game itself.
##### Task 1 :
*  Create a basic logical representation of the Hurkle's location and test if the system can correctly identify an incorrect guess with the simplified configuration

##### Task 2 :
*  Add a constraint that the Hurkle's location can be only in 1 tile at a time,using mutual exclusion and probabilistic logical principles.
  
##### Task 3 :
*  Add new symbols for all 8 directions: North, South , West, East, North-East, South-East, South-West, North-West
*  Using the coordinates and our knowledge for it, we can provide hints to the user that the Hurkle is at a location, particularly
*   Run one example check in your notebook. Your check for NW should call
the usual inference mechanism as shown below:
ask_if_true

#### Task 1 

In this simple environment, the knowledge base (KB) is only told about the true location of the Hurkle (H21). The inference system checks whether H42 is true (which it is not in our case). The KB is unable to deduce that the guess is false, because it simply lacks the necessary information to make this inference on its own. 


The tell() method is used to add the Hurkle's true location to the KB, and ask_if_true() is used to query whether a specific location (in this case, H42) is true or false. However, without additional constraints, the KB cannot infer that H42 must be false simply because H21 is true. 

This implementation demonstrates the limitations of a simple KB without mutual exclusion constraints, as it cannot make complex inferences about the Hurkle's location based solely on knowing one true location.

In [1]:
from logic import *

# Call the KB 
hurkleWorld = PropKB()

# Symbol for each possible position in the grid
H11, H21, H31, H41 = expr('H11'), expr('H21'), expr('H31'), expr('H41') #bottom row
H12, H22, H32, H42 = expr('H12'), expr('H22'), expr('H32'), expr('H42') 
H13, H23, H33, H43 = expr('H13'), expr('H23'), expr('H33'), expr('H43')
H14, H24, H34, H44 = expr('H14'), expr('H24'), expr('H34'), expr('H44') # top row 


# Hurkle Location 
Hurkle_location = H21

# We tell the KB the location
hurkleWorld.tell(Hurkle_location)

# Wrong Guess
guess = H42

# Is Hurkle's state guessed correctly?
hurkleWorld.ask_if_true(guess), hurkleWorld.ask_if_true(~guess)




(False, False)



#### Task 2 
In this Hurkle game environment, we implement a crucial constraint: the Hurkle can occupy only one square at a time. This **mutual exclusion principle** is encoded into our knowledge base (KB), significantly enhancing the inference system's capabilities.

The KB now contains not only the Hurkle's true location (e.g., `H21`) but also the logical implication that all other locations must be false. This is achieved through a **mutual exclusion clause** stating that if `H21` is true, then all other locations are false. Here’s a more detailed breakdown of how this works:

- **Logical Symbols for Constraints**: We use logical symbols, specifically **conjunction** (AND, `&`) and **negation** (NOT, `~`), to enforce the rule that the Hurkle can occupy only one square at a time.
  
- **Mutual Exclusion Clause**: This clause is constructed using negations to represent all locations other than the true Hurkle location. It effectively states that if the Hurkle is at one specific position, it cannot be at any other. This is expressed as:
  

 NOT location1 AND NOT location2


- **Implication Rule**: Using the implication symbol (`==>`), we add a rule to the knowledge base that asserts that if the Hurkle is in its true location, it must not be in any other location.

As a result, when we query the KB about any location (e.g., `H42`) using `ask_if_true()`, the system can accurately infer whether it is true or false. If `H21` is true, the KB can deduce that `H42` must be false, demonstrating the power of logical inference with properly defined constraints.





In [26]:
from logic import *

# Call the KB
hurkleWorld = PropKB()

# Symbol for each possible position in the grid
H11, H21, H31, H41 = expr('H11'), expr('H21'), expr('H31'), expr('H41')  # bottom row
H12, H22, H32, H42 = expr('H12'), expr('H22'), expr('H32'), expr('H42')
H13, H23, H33, H43 = expr('H13'), expr('H23'), expr('H33'), expr('H43')
H14, H24, H34, H44 = expr('H14'), expr('H24'), expr('H34'), expr('H44')  # top row
hurkle_locations = [H11, H21, H31, H41, H12, H22, H32, H42, H13, H23, H33, H43, H14, H24, H34, H44]

Hurkle_location = H21  

hurkleWorld.tell(Hurkle_location)

mutual_exclusion = [~loc for loc in hurkle_locations if loc != Hurkle_location]

mutual_exclusion_clause = mutual_exclusion[0]
for loc in mutual_exclusion[1:]:
    mutual_exclusion_clause &= loc

hurkleWorld.tell(Hurkle_location | '==>' | mutual_exclusion_clause)
for fact in hurkleWorld.clauses:
    print(fact)
# Make a guess
guess = H42
hurkleWorld.ask_if_true(guess), hurkleWorld.ask_if_true(~guess)




H21
(~H11 | ~H21)
(~H31 | ~H21)
(~H41 | ~H21)
(~H12 | ~H21)
(~H22 | ~H21)
(~H32 | ~H21)
(~H42 | ~H21)
(~H13 | ~H21)
(~H23 | ~H21)
(~H33 | ~H21)
(~H43 | ~H21)
(~H14 | ~H21)
(~H24 | ~H21)
(~H34 | ~H21)
(~H44 | ~H21)


(False, True)

#### Task 3: Implementing Directional Guidance in the Hurkle Game

With the mutual exclusion constraint defined, the player can now receive directional hints towards the Hurkle based on their guess location. We use **eight directional symbols** (N, S, W, E, NE, SE, SW, NW), represented using `expr()` and stored in a list for efficient access.

To determine the correct direction between the Hurkle's and player's locations, we implemented a `directions()` function. This function calculates the relative direction of the player's guess and then calls `tell_directions()`, a helper function that updates the knowledge base with the correct direction.

The `tell_directions()` function performs two key actions:

1. **Adding the Correct Direction**: It asserts the correct direction (e.g., SE) into the knowledge base.
2. **Applying Mutual Exclusion**: It uses a logical clause to state that if one direction is true, all others must be false. This is achieved with logical **AND** (`&`) and **negation** (`~`) operators, similar to the approach used in the previous task.

The system relies on logical inference to update and confirm directional information. By calling `ask_if_true()`, we can query if a specific direction is correct based on the knowledge base’s state. For instance, if the Hurkle is at location 'H21' and the player's guess is at 'H42', the system determines that **Southeast (SE)** is the correct direction, marks SE as true, and negates all other directions. This ensures that when queried, only SE is confirmed as accurate.

After setting the Hurkle's location and making a guess, we can view the updated knowledge base and check various directions using `ask_if_true()`. 


The implementation is efficient and avoids the **\(2^n\)** combinatorial explosion common in exhaustive constraint systems. Instead of generating all combinations, this method directly asserts a single true direction and negates the rest, making the logic scalable. 
Below is the code used for this task and an example of querying the knowledge base for directional hints.




In [2]:
# Symbols for direction
from logic import PropKB, expr

N, S, W, E = expr('N'), expr('S'), expr('W'), expr('E')
NE, SE, SW, NW = expr('NE'), expr('SE'), expr('SW'), expr('NW')
directions_list = [N, S, W, E, NE, SE, SW, NW]

hurkleWorld = PropKB()

def directions(hurkle_location, guess_location):
    hx, hy = int(hurkle_location[1]), int(hurkle_location[2])
    gx, gy = int(guess_location[1]), int(guess_location[2])
    
    print(f"Hurkle Location: ({hx}, {hy}), Guess Location: ({gx}, {gy})")

    def tell_directions(correct_direction):
        hurkleWorld.tell(correct_direction)
        incorrect_directions = [~direction for direction in directions_list if direction != correct_direction]
        

        mutual_exclusion_clause = expr(' & '.join([str(direction) for direction in incorrect_directions])) 

        hurkleWorld.tell(correct_direction | '==>' | mutual_exclusion_clause)

    # Determine the direction
    if hx < gx and hy == gy: 
        tell_directions(E)  # Move East
    elif hx > gx and hy == gy: 
        tell_directions(W)  # Move West
    elif hy < gy and hx == gx: 
        tell_directions(S)  # Move South
    elif hy > gy and hx == gx: 
        tell_directions(N)  # Move North
    elif hx < gx and hy < gy:
        tell_directions(SE)  # Move South-East
    elif hx < gx and hy > gy:
        tell_directions(NW)  # Move North-West
    elif hx > gx and hy < gy:
        tell_directions(SW)  # Move South-West
    elif hx > gx and hy > gy:
        tell_directions(NE)  # Move North-East

# Set locations
hurkle_location = 'H21'  # location of H
guess_location = 'H42'    # Guess Location
# Add directional rules based on locations
directions(hurkle_location, guess_location)

# Print current facts in the knowledge base
print("Current facts in Hurkle_kb:")
for fact in hurkleWorld.clauses:
    print(fact)

# Check NW direction and its negation
print("Is the Hurkle in the North-West?", hurkleWorld.ask_if_true(NW), hurkleWorld.ask_if_true(~NW))  

# Now let us check all other directions
for direction in [N, S, W, E, NE, SE, SW]:
    print(f"Is {direction} true?", hurkleWorld.ask_if_true(direction), hurkleWorld.ask_if_true(~direction))


Hurkle Location: (2, 1), Guess Location: (4, 2)
Current facts in Hurkle_kb:
SE
(~N | ~SE)
(~S | ~SE)
(~W | ~SE)
(~E | ~SE)
(~NE | ~SE)
(~SW | ~SE)
(~NW | ~SE)
Is the Hurkle in the North-West? False True
Is N true? False True
Is S true? False True
Is W true? False True
Is E true? False True
Is NE true? False True
Is SE true? True False
Is SW true? False True
